In [1]:
import warnings
warnings.filterwarnings('ignore')
import os

import tws_data 
import statsmodels.tsa.stattools as ts
import statsmodels.api as sm
from ib_insync import *
import numpy as np
import pandas as pd
import datetime as dt
import time
pd.options.display.float_format = '{:,.4f}'.format
%matplotlib inline
from scipy.stats import norm as norm
import matplotlib.pyplot as plt
import requests as req
from bs4 import BeautifulSoup as bs
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import math


In [2]:
os.chdir('D:\Options Data\IB Intraday')

def create_dfrow(curr_option):
    df_row = pd.DataFrame({'Symbol': curr_option.contract.localSymbol, 
                           'Type':curr_option.contract.right, 
                           'Bid': curr_option.bid, 
                           'Ask': curr_option.ask, 
                           'Volume': curr_option.volume, 
                           'Strike': curr_option.contract.strike,
                           'Expiry': dt.datetime.strptime(curr_option.contract.lastTradeDateOrContractMonth, '%Y%m%d'),
                           'bidIV': curr_option.bidGreeks.impliedVol, 
                           'bidDelta': curr_option.bidGreeks.delta,
                           'bidGamma': curr_option.bidGreeks.gamma, 
                           'bidVega': curr_option.bidGreeks.vega,
                           'bidTheta': curr_option.bidGreeks.theta, 
                           'askIV': curr_option.askGreeks.impliedVol, 
                           'askDelta': curr_option.askGreeks.delta,
                           'askGamma': curr_option.askGreeks.gamma, 
                           'askVega': curr_option.askGreeks.vega,
                           'askTheta': curr_option.askGreeks.theta,
                           'modelIV': curr_option.modelGreeks.impliedVol, 
                           'modelDelta': curr_option.modelGreeks.delta,
                           'modelGamma': curr_option.modelGreeks.gamma, 
                           'modelVega': curr_option.modelGreeks.vega,
                           'modelTheta': curr_option.modelGreeks.theta}, index = [0])
    return df_row

def time_remaining(weekly_options, curr_spx):
    typ = weekly_options.reset_index().loc[0,'Type']

    try:
        interest_rate = bs(req.get('https://www.marketwatch.com/investing/bond/tmubmusd01m?countrycode=bx','lxml').text, 'lxml')
        interest_rate = float(interest_rate.select_one('h3[class*="intraday__price"]').select_one('bg-quote').text)/100
    except:
        interest_rate = 0.02422

    weekly_options['a'] = interest_rate - (weekly_options['modelIV']**2)/2

    if typ == 'P':
        weekly_options['b'] = weekly_options['modelIV']*norm.ppf(-weekly_options['modelDelta'])
    if typ == 'C':
        weekly_options['b'] = -weekly_options['modelIV']*norm.ppf(weekly_options['modelDelta'])
    weekly_options['c'] = np.log(curr_spx/weekly_options['Strike'])

    weekly_options_otm = weekly_options[abs(weekly_options.modelDelta) <= 0.5]
    weekly_options_itm = weekly_options[abs(weekly_options.modelDelta) > 0.5]

    if typ == 'P':
        weekly_options_otm['time_remaining'] = ((-weekly_options_otm.b - 
                                                 np.sqrt(weekly_options_otm.b**2 - 
                                                         4*weekly_options_otm.a*weekly_options_otm.c))/(2*weekly_options_otm.a))**2
        weekly_options_itm['time_remaining'] = ((-weekly_options_itm.b + 
                                                 np.sqrt(weekly_options_itm.b**2 - 
                                                         4*weekly_options_itm.a*weekly_options_itm.c))/(2*weekly_options_itm.a))**2
    if typ == 'C':
        weekly_options_otm['time_remaining'] = ((-weekly_options_otm.b + 
                                                 np.sqrt(weekly_options_otm.b**2 - 
                                                         4*weekly_options_otm.a*weekly_options_otm.c))/(2*weekly_options_otm.a))**2
        weekly_options_itm['time_remaining'] = ((-weekly_options_itm.b - 
                                                 np.sqrt(weekly_options_itm.b**2 - 
                                                         4*weekly_options_itm.a*weekly_options_itm.c))/(2*weekly_options_itm.a))**2
    weekly_times = pd.concat([weekly_options_otm, 
                              weekly_options_itm], axis = 0)[['Expiry','time_remaining']].dropna().groupby('Expiry').mean()

    del weekly_options['a'], weekly_options['b'], weekly_options['c'], weekly_options_itm, weekly_options_otm

    return weekly_options.merge(weekly_times.reset_index(), on = 'Expiry')

def get_options(ticker, exchange, contractType, earliest_date, latest_date, curr_iv, stdevs, save_csv = False):
    if contractType == 'Stock':
        security = Stock(ticker,exchange)
    else:
        security = Index(ticker, exchange)

    ib.qualifyContracts(security)

    chains = ib.reqSecDefOptParams(security.symbol, '', security.secType, security.conId)

    chains_df = util.df(chains)
    chain = next(c for c in chains if c.tradingClass == ticker and c.exchange == 'SMART')

    all_expirations = sorted(exp for exp in chain.expirations)
    expirations_dates = [dt.datetime.strptime(x, '%Y%m%d') for x in all_expirations]

    expirations = list(filter(lambda x: x[0] >= earliest_date and x[0] <= latest_date, list(zip(expirations_dates,all_expirations))))
    dte_expirations = [(expir[0] - dt.datetime.today()).days for expir in expirations]
    expirations = [expir[1] for expir in expirations]

    ib.reqHeadTimeStamp(security, whatToShow='TRADES', useRTH=True)

    end_date = '' #'20100506 13:00:00 PST'
    duration = '1 D'

    sec_bars = ib.reqHistoricalData(
            security,
            endDateTime=end_date,
            durationStr=duration,
            barSizeSetting='1 min',
            whatToShow='TRADES',
            useRTH=True,
            formatDate=1)


    sec_df = util.df(sec_bars).set_index('date')
    sec_df.index = pd.to_datetime(sec_df.index)

    curr_sec_price = sec_df.reset_index().loc[len(sec_df) - 1, 'close']
    bounds_lst = [curr_sec_price*(stdevs*stdevs/(100*np.sqrt(365.25/dte))) for dte in dte_expirations]


    contracts_list = []

    for bounds, expiration in zip(bounds_lst, expirations):
#         strikes = [strike for strike in chain.strikes
#                    if strike % 5 == 0
#                    and curr_sec_price - bounds < strike < curr_sec_price + bounds]
        strikes = [strike for strike in chain.strikes
                   if curr_sec_price - bounds < strike < curr_sec_price + bounds]

        rights = ['P', 'C']

        curr_contracts = [Option(ticker, expiration, strike, right, 'SMART')
                          for right in rights
                          for strike in strikes]
        contracts_list += [curr_contracts]

    puts_list = []
    calls_list = []

    for contracts in contracts_list:
        ib.qualifyContracts(*contracts)
        sec_options = ib.reqTickers(*contracts)

        puts_rows = []
        calls_rows = []

        for curr_option in sec_options:
            try:
                curr_row = create_dfrow(curr_option)
                if curr_option.contract.right == 'P':
                    puts_rows.append(curr_row)
                elif curr_option.contract.right == 'C':
                    calls_rows.append(curr_row)
            except:
                continue

        puts = pd.concat(puts_rows).reset_index(drop = True).sort_values(['Expiry','Strike'])
        calls = pd.concat(calls_rows).reset_index(drop = True).sort_values(['Expiry','Strike'])
        puts = time_remaining(puts, curr_sec_price)
        calls = time_remaining(calls, curr_sec_price)
        puts_list.append(puts)
        calls_list.append(calls)
        time.sleep(5)

    puts_df = pd.concat(puts_list,axis = 0).reset_index(drop = True)
    calls_df = pd.concat(calls_list,axis = 0).reset_index(drop = True)

    if save_csv:
        curr_time = dt.datetime.now().strftime('%Y%m%d-%H-%M')
        puts_df.to_csv('ib_puts' + curr_time + '.csv')
        calls_df.to_csv('ib_calls' + curr_time + '.csv')
        
    return puts_df, calls_df, curr_sec_price

In [3]:
client_id = 11

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=client_id)

<IB connected to 127.0.0.1:7496 clientId=11>

In [27]:
exchange = 'CBOE'
contractType = 'Index'
earliest_date = dt.datetime(2019,5,23)
latest_date = dt.datetime(2019,6,20)

ticker = 'SPX'
curr_iv = 22
stdevs = 15
curr_puts, curr_calls, curr_price = get_options(ticker, exchange, contractType, earliest_date, latest_date, curr_iv, stdevs, save_csv = False)

Started to throttle requests
Error 200, reqId 108: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20190620', strike=4200.0, right='P', exchange='SMART')
Error 200, reqId 134: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20190620', strike=1175.0, right='P', exchange='SMART')
Error 100, reqId 9: Max rate of messages per second has been exceeded:max=50 rec=55 (1)
Error 200, reqId 390: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20190620', strike=4200.0, right='C', exchange='SMART')
Error 200, reqId 416: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20190620', strike=1175.0, right='C', exchange='SMART')
Stopped to throttle requests
Unknown contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20190620', st